In [2]:
import sys; sys.path.append('../')
# load selectpercentile
from sklearn.feature_selection import SelectPercentile, f_classif
from cascade import CascadeClassifier
import numpy as np
import os
import pickle as pkl
import time
from feature_extractor import FeatureExtractor

In [2]:
feature_extractor = FeatureExtractor(shape=(19,19),
                                     percentile=10,
                                     all_features_file='Features/all_features.npz',
                                     selected_features_file='Features/selected_features.npz',
                                     labels_file='Features/labels.npy',
                                     indecies_file='Features/indecies.npy')

In [3]:
indecies, X = feature_extractor.selectPercentile()
y = np.load('Features/labels.npy')
print(X.shape, y.shape)

(6396, 6977) (6977,)


In [10]:
# # do the following if the file selected_features is not present
# X, y, indecies = None, None, None
# if not os.path.exists('selected_features.npz'):
#     X = np.load('all_features.npz')['arr_0']
#     y = np.load('labels.npy')
#     selector = SelectPercentile(f_classif, percentile=10)
#     indecies = selector.fit(X.T, y).get_support(indices=True)
#     X = X[indecies, :]
#     np.savez('selected_features.npz', X)
#     # save indecies for later use
#     np.save('indecies.npy', indecies)
# else:
#     X = np.load('selected_features.npz')['arr_0']
#     y = np.load('labels.npy')
#     indecies = np.load('indecies.npy')
#     print(X.shape)
#     print(y.shape)

(6396, 6977)
(6977,)


In [11]:
# !rm -r StrongClassifier
# !rm cascadeClassifier.pkl

In [12]:
s_t = time.time()
cascadeClassifier= CascadeClassifier(X, y, batchsize=1000, verbose=False)
print(np.sum(y == 1))
print(np.sum(y == 0))
acc = cascadeClassifier.train()
print('accuracy: ', acc)
print('time taken: ', time.time() - s_t, 's')
# save the classifier
cascadeClassifier.save('Features/cascadeClassifier.pkl')

2429
4548
Continue training from layer 33 / 32
accuracy:  0.9635946681954995
time taken:  1.2926416397094727 s


In [9]:
s_t = time.time()
t_f_idx_map, t_features, t_labels = feature_extractor.extractFeaturesByIndecies(pos_path='face.test/test/face/',
                                            neg_path='face.test/test/non-face/',
                                            cascadeClassifier=cascadeClassifier)
print(t_features.shape, t_labels.shape)
predictions = cascadeClassifier.predict(t_features, t_f_idx_map)
print('test accuracy: ', np.sum(predictions == t_labels) / t_labels.shape[0])
print('time taken: ', time.time() - s_t, 's')

Now reading dataset...
Now extracting features from dataset...
(175, 24045) (24045,)
test accuracy:  0.9782491162403826
time taken:  0.8144936561584473 s


In [11]:
print(np.sum(t_labels == 1))
print(np.sum(t_labels == 0))
print(np.sum(predictions == 1))
print(np.sum(predictions == 0))


472
23573
145
23900
